In [11]:
import pandas as pd
train = pd.read_csv('train_jieba.csv')
train.head()

,ID,Age,Gender,Education,query,search_num,text_len,text_avg_len
0,22DD920316420BE2DF8D6EE651BA174B,1,0,4,柔和 双沟 女生 中财网 首页 <BLANK> 财经 http : / / pan . ba...,275,2380,8.654545
1,43CC3AF5A8D6430A3B572337A889AFE4,2,0,3,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦...",186,1644,8.838710
2,6931EFC26D229CCFCEA125D3F3C21E57,4,1,3,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 <BLANK> 写 范例 三维 绘图 软件...,173,1125,6.502890
3,E780470C3BB0D340334BD08CDCC3C71A,2,1,4,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...,117,1012,8.649573
4,0D58894CD481D4B3039000661665C0DB,3,1,3,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...,107,619,5.785047


In [2]:
from gensim.models import LdaModel, LdaMulticore
from gensim import corpora
texts = train['query'].values.tolist()
data = [[word for word in doc.split()] for doc in texts]
dct = corpora.Dictionary(data)
corpus = [dct.doc2bow(line) for line in data]

In [3]:
num_topics = 72
lda_model = LdaMulticore(corpus=corpus,
                             id2word=dct,
                             random_state=100,
                             num_topics=num_topics,
                             passes=10,
                             chunksize=1000,
                             batch=False,
                             alpha='asymmetric',
                             decay=0.5,
                             offset=64,
                             eta=None,
                             eval_every=0,
                             iterations=100,
                             gamma_threshold=0.001,
                             per_word_topics=True)
lda_model.save('model/lda_model.model')

FileNotFoundError: [Errno 2] No such file or directory: 'model/lda_model.model.state.sstats.npy'

In [4]:
lda_model.save('model/lda_model.model')

In [8]:
train_vecs = []
for i in range(len(train)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(num_topics)]
    train_vecs.append(topic_vec)
print(train_vecs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
del train_vecs

In [12]:
import numpy as np
train_lda = np.array(train_vecs)

# LDA + Logistic

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=666)
label_arr = {'Gender':2,'Age':6,'Education':6}
for label in label_arr.keys():
    for i,(train_index,eval_index) in enumerate(kf.split(train_lda)):
        print(len(train_index),len(eval_index))

        lb = LabelEncoder()
        train[label] = lb.fit_transform(train[label].tolist())
        #训练集
        X_train = train_lda[train_index]
        y_train = train[label][train_index]

        #验证集
        X_eval = train_lda[eval_index]
        y_eval = train[label][eval_index]

        model = LogisticRegression(C=4, dual=True) 
        model.fit(X_train,y_train)

        ####对于验证集进行预测
        train_matrix = np.zeros((train.shape[0],label_arr[label]))
        eval_prob = model.predict_proba(X_eval)
        train_matrix[eval_index] = eval_prob.reshape((X_eval.shape[0], label_arr[label]))#array

        eval_pred = np.argmax(eval_prob,axis=1)
        eval_pred = lb.inverse_transform(eval_pred)
        score = metrics.accuracy_score(lb.inverse_transform(y_eval),eval_pred)
        
        ###保存模型和数据
        
#         model.save
#         cv_scores.append(score)
        print("-----{}:第{}次-----".format(label,i))
        print("validation score is",score)

70793 17699


C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


-----Gender:第0次-----
validation score is 0.619526526922425
70793 17699


C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


-----Gender:第1次-----
validation score is 0.6153454997457484
70794 17698


...

-----Education:第2次-----
validation score is 0.4761554977963612
70794 17698


C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


-----Education:第3次-----
validation score is 0.4772855689908464
70794 17698


C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\LZP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


-----Education:第4次-----
validation score is 0.47802011526726185


# LDA + LGBM

In [20]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import numpy as np
import lightgbm as lgb
#sklearn api
kf = KFold(n_splits=5, shuffle=True, random_state=666)
label_arr = {'Gender':2,'Age':6,'Education':6}
# label_arr = {'Age':6,'Gender':2,'Education':6}
lgbm_class = {'Age':'multiclass','Gender':'binary','Education':'multiclass'}
lgbm_metric = {'Age':'multi_logloss','Gender':'binary_logloss','Education':'multi_logloss'}
all_score = []
for label in label_arr.keys():
    for i,(train_index,eval_index) in enumerate(kf.split(train_lda)):
        print(len(train_index),len(eval_index))

        lb = LabelEncoder()
        train[label] = lb.fit_transform(train[label].tolist())
        #训练集
        X_train = train_lda[train_index]
        y_train = train[label][train_index]

        #验证集
        X_eval = train_lda[eval_index]
        y_eval = train[label][eval_index]

        model =lgb.LGBMClassifier(boosting_type='gbdt', 
                   num_leaves=2**5,
                   max_depth=-1, 
                   learning_rate= 0.1,
                   n_estimators=2000,
                   objective=lgbm_class[label],
                   subsample=0.7,#
                   colsample_bytree=0.5,#
                   reg_lambda=10,#l2
                   n_jobs=16, #
#                    num_class=label_arr[label],#
                   silent=True, 
                   random_state=2019,
#                    class_weight=20,
                   colsample_bylevel=0.5,
                   min_child_weight=1.5,
                   metric=lgbm_metric[label]
                  )
        model.fit(X_train,y_train,eval_set=(X_eval,y_eval), early_stopping_rounds=50)

        ####对于验证集进行预测
        train_matrix = np.zeros((train.shape[0],label_arr[label]))
        eval_prob = model.predict_proba(X_eval)
#         print(eval_prob.tolist())
        print(eval_prob.shape)
#         train_matrix[eval_index] = eval_prob..reshape((X_eval.shape[0], label_arr[label]))#array
#         if label == 'Gender':
#             eval_prob = eval_prob[0].reshape((label_arr[label],X_eval.shape[0])).T
        eval_pred = np.argmax(eval_prob,axis=1)
        eval_pred = lb.inverse_transform(eval_pred)
        score = metrics.accuracy_score(lb.inverse_transform(y_eval),eval_pred)
        all_score.append([label,i,score])
        ###保存模型和数据
        
#         model.save
#         cv_scores.append(score)
        print("-----{}:第{}次-----".format(label,i))
        print("validation score is",score)
print(all_score)

70793 17699
[1]	valid_0's binary_logloss: 0.670258
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.664496
[3]	valid_0's binary_logloss: 0.660392
[4]	valid_0's binary_logloss: 0.655943
[5]	valid_0's binary_logloss: 0.650274
[6]	valid_0's binary_logloss: 0.645594
[7]	valid_0's binary_logloss: 0.643877
[8]	valid_0's binary_logloss: 0.639861
[9]	valid_0's binary_logloss: 0.638766
[10]	valid_0's binary_logloss: 0.636396
[11]	valid_0's binary_logloss: 0.634307
[12]	valid_0's binary_logloss: 0.631221
[13]	valid_0's binary_logloss: 0.630311
[14]	valid_0's binary_logloss: 0.628714
[15]	valid_0's binary_logloss: 0.627161
[16]	valid_0's binary_logloss: 0.624477
[17]	valid_0's binary_logloss: 0.623635
[18]	valid_0's binary_logloss: 0.622165
[19]	valid_0's binary_logloss: 0.620747
[20]	valid_0's binary_logloss: 0.619691
[21]	valid_0's binary_logloss: 0.618413
[22]	valid_0's binary_logloss: 0.617062
[23]	valid_0's binary_logloss: 0.616253
[24]	valid_0's b

[46]	valid_0's binary_logloss: 0.608707
[47]	valid_0's binary_logloss: 0.608554
[48]	valid_0's binary_logloss: 0.608427
[49]	valid_0's binary_logloss: 0.608324
[50]	valid_0's binary_logloss: 0.608274
[51]	valid_0's binary_logloss: 0.608219
[52]	valid_0's binary_logloss: 0.608138
[53]	valid_0's binary_logloss: 0.608109
[54]	valid_0's binary_logloss: 0.607956
[55]	valid_0's binary_logloss: 0.607917
[56]	valid_0's binary_logloss: 0.607817
[57]	valid_0's binary_logloss: 0.607782
[58]	valid_0's binary_logloss: 0.607746
[59]	valid_0's binary_logloss: 0.607701
[60]	valid_0's binary_logloss: 0.607618
[61]	valid_0's binary_logloss: 0.607533
[62]	valid_0's binary_logloss: 0.607475
[63]	valid_0's binary_logloss: 0.607471
[64]	valid_0's binary_logloss: 0.607443
[65]	valid_0's binary_logloss: 0.607437
[66]	valid_0's binary_logloss: 0.607415
[67]	valid_0's binary_logloss: 0.607358
[68]	valid_0's binary_logloss: 0.607357
[69]	valid_0's binary_logloss: 0.607328
[70]	valid_0's binary_logloss: 0.607293


[108]	valid_0's binary_logloss: 0.611625
[109]	valid_0's binary_logloss: 0.611628
[110]	valid_0's binary_logloss: 0.611636
[111]	valid_0's binary_logloss: 0.611658
[112]	valid_0's binary_logloss: 0.611674
[113]	valid_0's binary_logloss: 0.611694
[114]	valid_0's binary_logloss: 0.611677
[115]	valid_0's binary_logloss: 0.611673
[116]	valid_0's binary_logloss: 0.61169
[117]	valid_0's binary_logloss: 0.611683
[118]	valid_0's binary_logloss: 0.611664
[119]	valid_0's binary_logloss: 0.611694
Early stopping, best iteration is:
[69]	valid_0's binary_logloss: 0.61142
(17698, 2)
-----Gender:第2次-----
validation score is 0.6523901005763363
70794 17698
[1]	valid_0's binary_logloss: 0.669985
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.664171
[3]	valid_0's binary_logloss: 0.659949
[4]	valid_0's binary_logloss: 0.655353
[5]	valid_0's binary_logloss: 0.649586
[6]	valid_0's binary_logloss: 0.644855
[7]	valid_0's binary_logloss: 0.64317
[8]	valid_0's binar

[52]	valid_0's binary_logloss: 0.61054
[53]	valid_0's binary_logloss: 0.61056
[54]	valid_0's binary_logloss: 0.610455
[55]	valid_0's binary_logloss: 0.610392
[56]	valid_0's binary_logloss: 0.610287
[57]	valid_0's binary_logloss: 0.610234
[58]	valid_0's binary_logloss: 0.61019
[59]	valid_0's binary_logloss: 0.610171
[60]	valid_0's binary_logloss: 0.610127
[61]	valid_0's binary_logloss: 0.610131
[62]	valid_0's binary_logloss: 0.610061
[63]	valid_0's binary_logloss: 0.610023
[64]	valid_0's binary_logloss: 0.610001
[65]	valid_0's binary_logloss: 0.610006
[66]	valid_0's binary_logloss: 0.610006
[67]	valid_0's binary_logloss: 0.609992
[68]	valid_0's binary_logloss: 0.609982
[69]	valid_0's binary_logloss: 0.609965
[70]	valid_0's binary_logloss: 0.609931
[71]	valid_0's binary_logloss: 0.609952
[72]	valid_0's binary_logloss: 0.609963
[73]	valid_0's binary_logloss: 0.609962
[74]	valid_0's binary_logloss: 0.609916
[75]	valid_0's binary_logloss: 0.609941
[76]	valid_0's binary_logloss: 0.609914
[77

[123]	valid_0's multi_logloss: 1.25508
[124]	valid_0's multi_logloss: 1.25511
[125]	valid_0's multi_logloss: 1.25511
[126]	valid_0's multi_logloss: 1.25514
[127]	valid_0's multi_logloss: 1.25516
[128]	valid_0's multi_logloss: 1.25519
[129]	valid_0's multi_logloss: 1.25522
[130]	valid_0's multi_logloss: 1.25524
[131]	valid_0's multi_logloss: 1.25529
Early stopping, best iteration is:
[81]	valid_0's multi_logloss: 1.25364
(17699, 6)
-----Age:第0次-----
validation score is 0.4660715294649415
70793 17699
[1]	valid_0's multi_logloss: 1.35812
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 1.34509
[3]	valid_0's multi_logloss: 1.3343
[4]	valid_0's multi_logloss: 1.32415
[5]	valid_0's multi_logloss: 1.32049
[6]	valid_0's multi_logloss: 1.31289
[7]	valid_0's multi_logloss: 1.30546
[8]	valid_0's multi_logloss: 1.2994
[9]	valid_0's multi_logloss: 1.29473
[10]	valid_0's multi_logloss: 1.2907
[11]	valid_0's multi_logloss: 1.28805
[12]	valid_0's multi_logloss:

[66]	valid_0's multi_logloss: 1.24921
[67]	valid_0's multi_logloss: 1.24915
[68]	valid_0's multi_logloss: 1.2492
[69]	valid_0's multi_logloss: 1.24924
[70]	valid_0's multi_logloss: 1.24928
[71]	valid_0's multi_logloss: 1.24928
[72]	valid_0's multi_logloss: 1.24931
[73]	valid_0's multi_logloss: 1.24931
[74]	valid_0's multi_logloss: 1.24932
[75]	valid_0's multi_logloss: 1.24933
[76]	valid_0's multi_logloss: 1.24937
[77]	valid_0's multi_logloss: 1.24937
[78]	valid_0's multi_logloss: 1.24939
[79]	valid_0's multi_logloss: 1.2494
[80]	valid_0's multi_logloss: 1.24944
[81]	valid_0's multi_logloss: 1.24947
[82]	valid_0's multi_logloss: 1.24955
[83]	valid_0's multi_logloss: 1.2496
[84]	valid_0's multi_logloss: 1.24963
[85]	valid_0's multi_logloss: 1.24964
[86]	valid_0's multi_logloss: 1.24971
[87]	valid_0's multi_logloss: 1.24978
[88]	valid_0's multi_logloss: 1.24983
[89]	valid_0's multi_logloss: 1.24988
[90]	valid_0's multi_logloss: 1.24989
[91]	valid_0's multi_logloss: 1.24988
[92]	valid_0's 

[40]	valid_0's multi_logloss: 1.2544
[41]	valid_0's multi_logloss: 1.25422
[42]	valid_0's multi_logloss: 1.25399
[43]	valid_0's multi_logloss: 1.25373
[44]	valid_0's multi_logloss: 1.25358
[45]	valid_0's multi_logloss: 1.25342
[46]	valid_0's multi_logloss: 1.25323
[47]	valid_0's multi_logloss: 1.2531
[48]	valid_0's multi_logloss: 1.25296
[49]	valid_0's multi_logloss: 1.25289
[50]	valid_0's multi_logloss: 1.2528
[51]	valid_0's multi_logloss: 1.2527
[52]	valid_0's multi_logloss: 1.25267
[53]	valid_0's multi_logloss: 1.25261
[54]	valid_0's multi_logloss: 1.25257
[55]	valid_0's multi_logloss: 1.25252
[56]	valid_0's multi_logloss: 1.25249
[57]	valid_0's multi_logloss: 1.25242
[58]	valid_0's multi_logloss: 1.25239
[59]	valid_0's multi_logloss: 1.25238
[60]	valid_0's multi_logloss: 1.25233
[61]	valid_0's multi_logloss: 1.2523
[62]	valid_0's multi_logloss: 1.25234
[63]	valid_0's multi_logloss: 1.25234
[64]	valid_0's multi_logloss: 1.25234
[65]	valid_0's multi_logloss: 1.25237
[66]	valid_0's mu

[9]	valid_0's multi_logloss: 1.19127
[10]	valid_0's multi_logloss: 1.18777
[11]	valid_0's multi_logloss: 1.18418
[12]	valid_0's multi_logloss: 1.18083
[13]	valid_0's multi_logloss: 1.17756
[14]	valid_0's multi_logloss: 1.17488
[15]	valid_0's multi_logloss: 1.17277
[16]	valid_0's multi_logloss: 1.17086
[17]	valid_0's multi_logloss: 1.16931
[18]	valid_0's multi_logloss: 1.16821
[19]	valid_0's multi_logloss: 1.16681
[20]	valid_0's multi_logloss: 1.16553
[21]	valid_0's multi_logloss: 1.16458
[22]	valid_0's multi_logloss: 1.16345
[23]	valid_0's multi_logloss: 1.16242
[24]	valid_0's multi_logloss: 1.16162
[25]	valid_0's multi_logloss: 1.16057
[26]	valid_0's multi_logloss: 1.15983
[27]	valid_0's multi_logloss: 1.15933
[28]	valid_0's multi_logloss: 1.15863
[29]	valid_0's multi_logloss: 1.1584
[30]	valid_0's multi_logloss: 1.15785
[31]	valid_0's multi_logloss: 1.15742
[32]	valid_0's multi_logloss: 1.15712
[33]	valid_0's multi_logloss: 1.1567
[34]	valid_0's multi_logloss: 1.15635
[35]	valid_0's 

[101]	valid_0's multi_logloss: 1.16126
[102]	valid_0's multi_logloss: 1.16127
[103]	valid_0's multi_logloss: 1.16134
[104]	valid_0's multi_logloss: 1.16138
[105]	valid_0's multi_logloss: 1.16144
[106]	valid_0's multi_logloss: 1.16147
[107]	valid_0's multi_logloss: 1.16149
[108]	valid_0's multi_logloss: 1.16152
[109]	valid_0's multi_logloss: 1.16158
[110]	valid_0's multi_logloss: 1.16164
[111]	valid_0's multi_logloss: 1.16177
[112]	valid_0's multi_logloss: 1.16185
[113]	valid_0's multi_logloss: 1.16188
[114]	valid_0's multi_logloss: 1.16192
Early stopping, best iteration is:
[64]	valid_0's multi_logloss: 1.16007
(17698, 6)
-----Education:第2次-----
validation score is 0.4833879534410668
70794 17698
[1]	valid_0's multi_logloss: 1.24876
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 1.24029
[3]	valid_0's multi_logloss: 1.23139
[4]	valid_0's multi_logloss: 1.2224
[5]	valid_0's multi_logloss: 1.21594
[6]	valid_0's multi_logloss: 1.21234
[7]	valid_0's

[81]	valid_0's multi_logloss: 1.15129
[82]	valid_0's multi_logloss: 1.15135
[83]	valid_0's multi_logloss: 1.1514
[84]	valid_0's multi_logloss: 1.15143
[85]	valid_0's multi_logloss: 1.15146
[86]	valid_0's multi_logloss: 1.15145
[87]	valid_0's multi_logloss: 1.15145
[88]	valid_0's multi_logloss: 1.15146
[89]	valid_0's multi_logloss: 1.15147
[90]	valid_0's multi_logloss: 1.15146
[91]	valid_0's multi_logloss: 1.15149
[92]	valid_0's multi_logloss: 1.15158
[93]	valid_0's multi_logloss: 1.15161
[94]	valid_0's multi_logloss: 1.15164
[95]	valid_0's multi_logloss: 1.15163
[96]	valid_0's multi_logloss: 1.15169
[97]	valid_0's multi_logloss: 1.15172
[98]	valid_0's multi_logloss: 1.15177
[99]	valid_0's multi_logloss: 1.15185
[100]	valid_0's multi_logloss: 1.15188
[101]	valid_0's multi_logloss: 1.15194
[102]	valid_0's multi_logloss: 1.15198
[103]	valid_0's multi_logloss: 1.15202
[104]	valid_0's multi_logloss: 1.15206
[105]	valid_0's multi_logloss: 1.15208
[106]	valid_0's multi_logloss: 1.15211
[107]	

In [21]:
pd.DataFrame(all_score)

,0,1,2
0,Gender,0,0.657890
1,Gender,1,0.660602
2,Gender,2,0.652390
3,Gender,3,0.658662
4,Gender,4,0.655102
5,Age,0,0.466072
6,Age,1,0.471609
7,Age,2,0.469714
8,Age,3,0.469206
9,Age,4,0.472426
